In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import html
import re
import random
import json

In [2]:
def create_dict(tag, attrs, text, re_pattern):
    return {
        "tag": tag,
        "attrs": attrs,
        "text": text,
        "re_pattern": re_pattern,
    }


book_link_from_html = create_dict(
    tag="h4",
    attrs={"class": "product-name-title"},
    text="",
    re_pattern="",
)

df = pd.DataFrame(columns=["id", "title", "url"])

In [3]:

def req_to_address(method, url, headers, params):
    try:
        response = requests.request(
            method=method,
            url=url,
            headers=headers,
            params=params,
        )
        if response.status_code == 200:
            return response.content
        else:
            print("Failed to fetch the website with url:", url)
            return None
    except:
        print("Error in get request with url:", url)

In [4]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15"
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
]
headers = {
    "User-Agent": random.choice(user_agents),
    "Accept-Language": "en-US,en;q=0.9,fa;q=0.8",
}

main_url = "https://www.iranketab.ir"
# category1_url = "tag/103-fiction"
book_url = "/book?"
# page = "?Page="
maximum_number_books = 1000
page_number = 1

while True:
    query_params = {
        "pagenumber": str(page_number),
        "pagesize": str(maximum_number_books),
        "sortOrder": "date_asc",
        # "tagid": "103",
    }
    html_content = req_to_address(
        method="GET",
        url=main_url + book_url,
        headers=headers,
        params=query_params,
    )
    soup = BeautifulSoup(html_content, "html.parser")
    elements = soup.find_all(
        book_link_from_html["tag"], attrs=book_link_from_html["attrs"]
    )

    cnt = 0
    for element in elements:
        a_element = element.find("a")
        if a_element:
            link = a_element["href"]
            start = link.find("/book/") + len("/book/")
            end = link.find("-")
            df.loc[len(df)] = {
                "id": link[start:end],
                "title": a_element.text.strip(),
                "url": main_url + link,
            }
            cnt += 1

    if cnt != maximum_number_books:
        print("break")
        break

    page_number += 1
    headers["User-Agent"] = random.choice(user_agents)

break


In [23]:
df.to_csv("URL_address_books.csv", index=False, encoding="utf-8-sig")

In [39]:
df.describe()

,id,title,url
count,99768,99768,99768
unique,99768,92991,99768
top,43,دفتر یادداشت,https://www.iranketab.ir/book/43-a-fraction-of...
freq,1,542,1
